In [16]:
%run utils.ipynb

In [2]:
tech = ['HCLTECH', 'WIPRO', 'INFY', 'TCS', 'TECHM']
banks = ['HDFCBANK', 'ICICIBANK', 'SBIN', 'KOTAKBANK', 'AXISBANK', 'FEDERALBNK', 'KARURVYSYA', 'CANBK']
others = ['RELIANCE', 'HINDUNILVR', 'ITC', 'M&M', 'TATAMOTORS', 'TATASTEEL', 'CIPLA']

_all = tech + banks + others

nifty_50_stocks = ['EICHERMOT','HEROMOTOCO','NESTLEIND','ONGC','BAJAJ-AUTO','TATASTEEL','GRASIM',
                   'BRITANNIA','BAJFINANCE','DIVISLAB','M&M','HINDUNILVR','HDFCBANK','HDFCLIFE','BHARTIARTL','TCS',
                   'LT','DRREDDY','ULTRACEMCO','SUNPHARMA','NTPC','TATAMOTORS','UPL','SBIN','HINDALCO','ITC','JSWSTEEL',
                   'COALINDIA','RELIANCE','BPCL','LTIM','MARUTI','HCLTECH','POWERGRID','WIPRO','SBILIFE','AXISBANK',
                   'ADANIPORTS','ICICIBANK','TITAN','BAJAJFINSV','KOTAKBANK','TATACONSUM','APOLLOHOSP','INFY','ASIANPAINT',
                   'ADANIENT','INDUSINDBK','TECHM','CIPLA',]


In [3]:
final_df = pd.concat([analyse(i, save_to_disk=True)[0] for i in nifty_50_stocks]).reset_index(drop=True)

In [4]:
final_df

,10_cross_30,MACD_Signal_MACD,MACD_lim,abv_50,abv_200,actual_returns,proba,stock
0,1,0,0,0,0,3.43,0.76,EICHERMOT
1,1,0,1,0,0,2.89,0.83,EICHERMOT
2,1,0,0,1,0,2.79,0.72,EICHERMOT
3,0,1,0,1,1,0.53,1.00,HEROMOTOCO
4,0,1,1,0,0,2.54,0.83,NESTLEIND
...,...,...,...,...,...,...,...,...
162,0,1,1,1,1,2.72,0.66,ADANIENT
163,0,1,0,1,0,8.35,1.00,INDUSINDBK
164,1,1,0,1,0,7.83,1.00,INDUSINDBK
165,0,1,1,0,0,2.37,0.78,TECHM


In [5]:
summary = final_df.groupby(['10_cross_30', 'MACD_Signal_MACD', 'MACD_lim', 'abv_50', 'abv_200']).agg(
    avg_return = ('actual_returns', 'mean'),
    median_return = ('actual_returns', 'median'),
    avg_probability = ('proba', 'mean'),
    stocks = ('stock', 'unique'),
    stock_count = ('stock', 'count')
)

summary.sort_values(by=['stock_count', 'avg_probability', 'avg_return'], ascending=False)

avg_return  \
10_cross_30 MACD_Signal_MACD MACD_lim abv_50 abv_200               
1           0                1        0      0          3.266471   
0           1                0        1      0          4.388571   
1           0                1        0      1          2.516154   
0           1                0        1      1          4.053333   
                             1        1      0          2.801818   
1           0                0        0      0          2.305455   
            1                0        0      1          2.437273   
                                             0          1.939000   
                                      1      0          4.900000   
            0                0        0      1          1.926667   
0           1                1        1      1          1.762222   
1           0                0        1      0          2.085000   
                                             1          1.544286   
0           1                1        0      0          2.946667   
                                             1          1.496667   
1           1                1        0      0          2.724000   
                             0        1      1          5.962500   
            0                1        1      0          2.005000   
            1                1        1      0          2.240000   
                                      0      1          1.990000   
0           1                0        0      0          2.260000   

                                                      median_return  \
10_cross_30 MACD_Signal_MACD MACD_lim abv_50 abv_200                  
1           0                1        0      0                3.120   
0           1                0        1      0                4.465   
1           0                1        0      1                2.300   
0           1                0        1      1                3.900   
                             1        1      0                2.090   
1           0                0        0      0                2.070   
            1                0        0      1                2.040   
                                             0                1.680   
                                      1      0                5.110   
            0                0        0      1                1.640   
0           1                1        1      1                1.770   
1           0                0        1      0                2.010   
                                             1                0.790   
0           1                1        0      0                2.680   
                                             1                1.435   
1           1                1        0      0                2.250   
                             0        1      1                6.175   
            0                1        1      0                2.005   
            1                1        1      0                2.240   
                                      0      1                1.990   
0           1                0        0      0                2.260   

                                                      avg_probability  \
10_cross_30 MACD_Signal_MACD MACD_lim abv_50 abv_200                    
1           0                1        0      0               0.828824   
0           1                0        1      0               0.970000   
1           0                1        0      1               0.727692   
0           1                0        1      1               0.941667   
                             1        1      0               0.806364   
1           0                0        0      0               0.761818   
            1                0        0      1               0.735455   
                                             0               0.688000   
                                      1      0               0.920000   
            0                0        0      1      

In [6]:
signal_grade = pd.qcut(summary['stock_count'].rename('signal_grade'), 4, labels=['grade D', 'grade C', 'grade B', 'grade A'])

summary = summary.merge(signal_grade, how='left', left_index=True, right_index=True)

In [7]:
summary = summary[summary['signal_grade'].isin(['grade A', 'grade B', 'grade C'])]

summary['expected_return'] = round(0.5 * (summary['avg_return'] + summary['median_return']), 3)

summary

avg_return  \
10_cross_30 MACD_Signal_MACD MACD_lim abv_50 abv_200               
0           1                0        1      0          4.388571   
                                             1          4.053333   
                             1        0      0          2.946667   
                                             1          1.496667   
                                      1      0          2.801818   
                                             1          1.762222   
1           0                0        0      0          2.305455   
                                             1          1.926667   
                                      1      0          2.085000   
                                             1          1.544286   
                             1        0      0          3.266471   
                                             1          2.516154   
            1                0        0      0          1.939000   
                                             1          2.437273   
                                      1      0          4.900000   

                                                      median_return  \
10_cross_30 MACD_Signal_MACD MACD_lim abv_50 abv_200                  
0           1                0        1      0                4.465   
                                             1                3.900   
                             1        0      0                2.680   
                                             1                1.435   
                                      1      0                2.090   
                                             1                1.770   
1           0                0        0      0                2.070   
                                             1                1.640   
                                      1      0                2.010   
                                             1                0.790   
                             1        0      0                3.120   
                                             1                2.300   
            1                0        0      0                1.680   
                                             1                2.040   
                                      1      0                5.110   

                                                      avg_probability  \
10_cross_30 MACD_Signal_MACD MACD_lim abv_50 abv_200                    
0           1                0        1      0               0.970000   
                                             1               0.941667   
                             1        0      0               0.770000   
                                             1               0.693333   
                                      1      0               0.806364   
                                             1               0.698889   
1           0                0        0      0               0.761818   
                                             1               0.746667   
                                      1      0               0.760000   
                                             1               0.715714   
                             1        0      0               0.828824   
                                             1               0.727692   
            1                0        0      0               0.688000   
                                             1               0.735455   
                                      1      0               0.920000   

                                                                                                 stocks  \
10_cross_30 MACD_Signal_MACD MACD_lim abv_50 abv_200                                                      
0           1                0        1      0        [TATASTEEL, BRITANNIA, HINDUNILVR, HDFCLIFE, B...   
                                             1        [HEROMOTOCO, DRREDDY, ULTRACEMCO, NTPC, SBIN, ...   
                   

In [8]:
cols = list(summary.index.names) + ['signal_grade', 'expected_return']

signals = summary.reset_index()[cols]

In [9]:
signals

,10_cross_30,MACD_Signal_MACD,MACD_lim,abv_50,abv_200,signal_grade,expected_return
0,0,1,0,1,0,grade A,4.427
1,0,1,0,1,1,grade A,3.977
2,0,1,1,0,0,grade C,2.813
3,0,1,1,0,1,grade C,1.466
4,0,1,1,1,0,grade B,2.446
5,0,1,1,1,1,grade C,1.766
6,1,0,0,0,0,grade B,2.188
7,1,0,0,0,1,grade C,1.783
8,1,0,0,1,0,grade C,2.048
9,1,0,0,1,1,grade C,1.167


In [10]:
TRADES = []
trade_in_progress = False
check_trend_change = False
holding_period = 10

for i in nifty_50_stocks:
    # _df = get_stock_info(i)
    _df = read_stock_info_from_disk(i)
    _df = add_signal_indicators(_df)

    _df['stock'] = i
    _df.reset_index().reset_index().apply(backtest_signals, axis=1)
    print(f'done performing backtesting for {i}')
    del _df
    trade_in_progress = False

done performing backtesting for EICHERMOT
done performing backtesting for HEROMOTOCO
done performing backtesting for NESTLEIND
done performing backtesting for ONGC
done performing backtesting for BAJAJ-AUTO
done performing backtesting for TATASTEEL
done performing backtesting for GRASIM
done performing backtesting for BRITANNIA
done performing backtesting for BAJFINANCE
done performing backtesting for DIVISLAB
done performing backtesting for M&M
done performing backtesting for HINDUNILVR
done performing backtesting for HDFCBANK
done performing backtesting for HDFCLIFE
done performing backtesting for BHARTIARTL
done performing backtesting for TCS
done performing backtesting for LT
done performing backtesting for DRREDDY
done performing backtesting for ULTRACEMCO
done performing backtesting for SUNPHARMA
done performing backtesting for NTPC
done performing backtesting for TATAMOTORS
done performing backtesting for UPL
done performing backtesting for SBIN
done performing backtesting for H

In [11]:
pd.DataFrame(TRADES).groupby('stock')['returns'].agg(['mean','median','sum','count']).reset_index()

,stock,mean,median,sum,count
0,ADANIENT,8.562442,1.4995,890.494,104
1,ADANIPORTS,1.403719,2.3100,134.757,96
2,APOLLOHOSP,1.075094,0.7870,149.438,139
3,ASIANPAINT,2.063376,1.4390,257.922,125
4,AXISBANK,0.615099,0.4090,80.578,131
5,BAJAJ-AUTO,0.917780,0.8680,100.038,109
6,BAJAJFINSV,1.118288,1.0050,124.130,111
7,BAJFINANCE,1.658000,2.2800,140.930,85
8,BHARTIARTL,1.146261,0.6280,136.405,119
9,BPCL,0.550776,-0.0095,73.804,134


In [12]:
pd.DataFrame(TRADES)['returns'].agg(['mean','median','sum','count']).reset_index()

,index,returns
0,mean,1.32033
1,median,0.84400
2,sum,7717.33000
3,count,5845.00000


In [13]:
TRADES_WITHOUT_TREND_CHECK = [_ for _ in TRADES]

In [17]:
TRADES = []
trade_in_progress = False
check_trend_change = True
holding_period = 10

for i in nifty_50_stocks:
    # _df = get_stock_info(i)
    _df = read_stock_info_from_disk(i)
    _df = add_signal_indicators(_df)

    _df['stock'] = i
    _df.reset_index().reset_index().apply(backtest_signals, axis=1)
    print(f'done performing backtesting for {i}')
    del _df
    trade_in_progress = False

done performing backtesting for EICHERMOT
done performing backtesting for HEROMOTOCO
done performing backtesting for NESTLEIND
done performing backtesting for ONGC
done performing backtesting for BAJAJ-AUTO
done performing backtesting for TATASTEEL
done performing backtesting for GRASIM
done performing backtesting for BRITANNIA
done performing backtesting for BAJFINANCE
done performing backtesting for DIVISLAB
done performing backtesting for M&M
done performing backtesting for HINDUNILVR
done performing backtesting for HDFCBANK
done performing backtesting for HDFCLIFE
done performing backtesting for BHARTIARTL
done performing backtesting for TCS
done performing backtesting for LT
done performing backtesting for DRREDDY
done performing backtesting for ULTRACEMCO
done performing backtesting for SUNPHARMA
done performing backtesting for NTPC
done performing backtesting for TATAMOTORS
done performing backtesting for UPL
done performing backtesting for SBIN
done performing backtesting for H

In [18]:
pd.DataFrame(TRADES).groupby('stock')['returns'].agg(['mean','median','sum','count']).reset_index()

,stock,mean,median,sum,count
0,ADANIENT,7.044971,-2.0045,972.206,138
1,ADANIPORTS,1.271306,-1.5270,153.828,121
2,APOLLOHOSP,0.478090,-1.6975,79.363,166
3,ASIANPAINT,1.250875,-0.1280,180.126,144
4,AXISBANK,0.726359,-1.7260,121.302,167
5,BAJAJ-AUTO,0.578410,-0.4030,77.507,134
6,BAJAJFINSV,2.150655,-1.8470,298.941,139
7,BAJFINANCE,1.223991,-1.9875,129.743,106
8,BHARTIARTL,0.744101,-1.8210,110.871,149
9,BPCL,1.000279,-1.9275,172.048,172


In [19]:
pd.DataFrame(TRADES)['returns'].agg(['mean','median','sum','count']).reset_index()

,index,returns
0,mean,1.051805
1,median,-1.694000
2,sum,7466.761000
3,count,7099.000000


In [ ]:
TRADES_WITH_TREND_CHECK = [_ for _ in TRADES]

In [20]:
all_trades = pd.DataFrame(TRADES_WITHOUT_TREND_CHECK)

In [22]:
x = all_trades[(all_trades['sell_price']=='') & (all_trades['returns']==0)].sort_values(by='buy_date', ascending=False)

x

,grade,buy_date,buy_index,sell_date,sell_index,buy_price,sell_price,returns,stock,expected_return
250,grade A,2023-09-28,5278,,5288,2970.000000,,0.0,HEROMOTOCO,2.408
2188,grade C,2023-09-28,5248,,5258,8154.350098,,0.0,ULTRACEMCO,1.167
2685,grade A,2023-09-28,5279,,5289,605.000000,,0.0,UPL,3.193
2824,grade A,2023-09-28,5929,,5939,589.950012,,0.0,SBIN,2.408
4104,grade C,2023-09-28,5931,,5941,405.350006,,0.0,WIPRO,1.167
4143,grade C,2023-09-28,1481,,1491,1289.400024,,0.0,SBILIFE,1.167
2567,grade A,2023-09-26,5929,,5939,619.799988,,0.0,TATAMOTORS,2.408
4493,grade A,2023-09-25,5275,,5285,956.349976,,0.0,ICICIBANK,2.408
1597,grade A,2023-09-22,1445,,1455,649.450012,,0.0,HDFCLIFE,2.408
3224,grade C,2023-09-22,5054,,5064,774.049988,,0.0,JSWSTEEL,1.783


In [23]:
current_price_data = {i: get_curr_price(i) for i in list(x.stock.unique())}

In [25]:
current_price_data

{'HEROMOTOCO': 2970.0,
 'ULTRACEMCO': 8154.35,
 'UPL': 605.0,
 'SBIN': 589.95,
 'WIPRO': 405.35,
 'SBILIFE': 1289.4,
 'TATAMOTORS': 614.1,
 'ICICIBANK': 942.6,
 'HDFCLIFE': 634.7,
 'JSWSTEEL': 768.1,
 'ASIANPAINT': 3170.35,
 'ITC': 440.75,
 'BRITANNIA': 4524.55,
 'BPCL': 343.95,
 'KOTAKBANK': 1733.2}

In [28]:
x['current_price'] = x['stock'].map(current_price_data)

x['current_returns'] = round(((x['current_price'] - x['buy_price'])/x['buy_price'])*100, 2)

x

,grade,buy_date,buy_index,sell_date,sell_index,buy_price,sell_price,returns,stock,expected_return,current_price,current_returns
250,grade A,2023-09-28,5278,,5288,2970.000000,,0.0,HEROMOTOCO,2.408,2970.00,0.00
2188,grade C,2023-09-28,5248,,5258,8154.350098,,0.0,ULTRACEMCO,1.167,8154.35,-0.00
2685,grade A,2023-09-28,5279,,5289,605.000000,,0.0,UPL,3.193,605.00,0.00
2824,grade A,2023-09-28,5929,,5939,589.950012,,0.0,SBIN,2.408,589.95,-0.00
4104,grade C,2023-09-28,5931,,5941,405.350006,,0.0,WIPRO,1.167,405.35,-0.00
4143,grade C,2023-09-28,1481,,1491,1289.400024,,0.0,SBILIFE,1.167,1289.40,-0.00
2567,grade A,2023-09-26,5929,,5939,619.799988,,0.0,TATAMOTORS,2.408,614.10,-0.92
4493,grade A,2023-09-25,5275,,5285,956.349976,,0.0,ICICIBANK,2.408,942.60,-1.44
1597,grade A,2023-09-22,1445,,1455,649.450012,,0.0,HDFCLIFE,2.408,634.70,-2.27
3224,grade C,2023-09-22,5054,,5064,774.049988,,0.0,JSWSTEEL,1.783,768.10,-0.77


In [29]:
x['current_returns'].sum()

-16.659999999999997